# Tile tensor basics


## Introduction
This notebook introduces the basics of using packing agnostic programming with [HELayers](https://ibm.github.io/helayers/). Specifically, how to include the [pyhelayers](https://pypi.org/project/pyhelayers/) library, initialize an encryption context and perform some simple encryption and decryption operations. 
HELayers and the concept of tile tensors rely on the paper

> Ehud Aharoni, Allon Adir, Moran Baruch, Nir Drucker, Gilad Ezov, Ariel Farkash, Lev Greenberg, Ramy Masalha, Guy Moshkowich, Dov Murik, Hayim Shaul, and Omri Soceanu. 2020. HeLayers: A Tile Tensors Framework for Large Neural Networks on Encrypted Data. Privacy Enhancing Technology Symposium (PETs) 2023, [Read more](https://petsymposium.org/popets/2023/popets-2023-0020.php)

See also an overview here: [tile tensors](https://ibm.github.io/helayers/user/tile_tensors.html).

<details>

<summary>Press for other literature on HELayers and tile tensors</summary>

#### Packing

* Ehud Aharoni, Moran Baruch, Nir Drucker, Gilad Ezov, Eyal Kushnir, Guy Moshkowich, and Omri Soceanu. 2022. Poster: Secure SqueezeNet inference in 4 minutes. 43rd IEEE Symposium on Security and Privacy (2022). [Read more](https://arxiv.org/abs/2011.01805)
* Ehud Aharoni, Nir Drucker, Gilad Ezov, Hayim Shaul, and Omri Soceanu. 2022. Complex Encoded Tile Tensors: Accelerating Encrypted Analytics. IEEE Security & Privacy 01, 2–10.
[Read more](https://doi.org/10.1109/MSEC.2022.3181689)

#### HELayers

* John Buselli. 2021. Secure AI workloads using fully homomorphic encrypted data. (Sep 2021). [Read more](https://developer.ibm.com/blogs/secure-ai-workloads-using-fully-homomorphic-encrypted-data/)
* IBM. 2021. Fully Homomorphic Encryption (FHE) - Never decrypt your data, even during computation. [Read more](https://fhe-website.mybluemix.net/?_ga=2.94032030.24152571.1666765225-1379575878.1644260568)

</details>

<br/>This tutorial assumes you are already familiar with the concept of homomorphic encryption (HE), with the CKKS scheme and the operations it supports. 

<details>

<summary>If not, press the button for relevant references</summary>

* Cheon, J. H., Kim, A., Kim, M., and Song, Y. (2017). Homomorphic Encryption for Arithmetic of Approximate Numbers. In T. Takagi and T. Peyrin (Eds.), Advances in Cryptology -- ASIACRYPT 2017 (Vol. 1, pp. 409–437). Springer International Publishing. [Read more](https://doi.org/10.1007/978-3-319-70694-8_15)

* Cheon, J. H., Han, K., Kim, A., Kim, M., and Song, Y. (2019). A Full RNS Variant of Approximate Homomorphic Encryption. In C. Cid and M. J. Jacobson Jr. (Eds.), Selected Areas in Cryptography -- SAC 2018 (pp. 347–368). Springer International Publishing. [Read more](https://doi.org/10.1007/978-3-030-10970-7_16)
    
</details>


<br>  

We start by importing the required packages for this tutorial

In [ ]:
import numpy as np
import math as math

## Step 1. Importing and initializing

Pyhelayers library is the Python API for HELayers, which we first need to import. 

For demo purposes, we turn on pretty printing of tiles.

In [ ]:
import pyhelayers

print("Imported pyhelayers", pyhelayers.VERSION)

pyhelayers.get_print_options().tt_demo_tiles = True

To use pyhelayers we first need to initialize `HeContext` and a `TTEncoder` object.  
(NOTE: to get help on pyhelayers or any of its classes type `help(pyhelayers)` or `help(pyhelayers.HeContext)`, etc...)

An HeContext is the main object for accessing the underlying FHE encryption scheme. Once initialized, it generates the keys and all the other data structures required for working with FHE. It is an abstract class with several different concrete implementations, each for a different FHE scheme, or a different open source library implementing these schemes.

In this notebook, we will use the `MockupContext`, which is a non-encrypted implementation for quick tests. However, HELayer's APIs abstract away the underlying details, so the same code can later be switched to work with a real FHE scheme by simply changing the HeContext creation.

An `HeContext` is initialized with an `HeConfigRequirement` object that defines our requirements from the FHE scheme: e.g., the number of slots in a ciphertext, the maximal multiplication depth of the evaluated circuit, or the security level. For demonstration purposes, we use ciphertexts with only 8 slots. To this end, we create a configuration using the `insecure` method, which explicitly states that the security level is 0. This allows us to use the Mockup implementation.

See the FHE basics demo for more details.

Finally, a `TTEncoder` object is responsible for encrypting and decrypting tensors. How it works is the central focus of these demos. Note that in FHE there's also an encoding step which is a preprocessing step before encryption, and similarly a postprocessing decoding step after decryption. Hence, the encryption and decryption methods are called `encode_encrypt` and `decrypt_decode`. In some advanced scenarios, it is useful to perform these steps separately.

In [ ]:
# Initialize a mockup context with 8 slots. 
requirement = pyhelayers.HeConfigRequirement.insecure(
    num_slots = 8,                  # Number of slots per ciphertext
    multiplication_depth = 2,       # Allow 2 levels of multiplications
    fractional_part_precision = 40, # Set the precision to 1/2^40.
    integer_part_precision = 20    # Set the largest number to 2^20.
    )

# Use a mockup HE context and initialize it with the requirements object above.
he_context = pyhelayers.MockupContext()
he_context.init(requirement)

# Create the Encoder using the context.
encoder = pyhelayers.TTEncoder(he_context)

print("Initialized a context with ", he_context.slot_count(), " slots and an encoder object")


A $5 \times 6$ matrix $M$ will serve as the plaintext for the demonstrations.



In [ ]:
def genDemoMatrix(m, n):
    """ Return an m x n matrix with values [0 ,..., m x n - 1].
    """
    return np.arange(0, m*n).reshape((m, n))

# Prepare a 5x6 matrix with values [0,...,29].
M = genDemoMatrix(5,6)
print("Plaintext matrix M=\n", M)

<br>

## Step 2. Different packings for the same matrix

Different packings affect the performance characteristic of every HE solution. 
We now show how the same marix can be packed in different ways:
<ul>
    <li> Packing each row of $M$ in a ciphertext</li>
    <li> Packing each column of $M$ in a ciphertext</li>
    <li> Using a more flexible packing</li>
</ul>

### Step 2.1. Packing each row of M in a ciphertext
We pack $M$ using tiles (ciphertexts) of shape $1 \times 8$. This effectively places a single row in every ciphertext.
Since we have 8 slots and each row has only 6 elements we end up with 8 - 6 = 2 unused slots in each ciphertext.

To perform the packing we use the [TTShape object](https//add.a.link). This objects keeps a shape.
Subsequently, we ask the encoder to encode and encrypt a tile tensor of M with the shape that we have just defined.

When printing the resulting ciphertext $C$, we see the tile tensor shape $\left[5, \frac{6}{8}\right]$ the chain index (CI), and the arrangement of the data inside 5 different tiles, i.e, the external tensor shape is $5 \times 1$.

In [ ]:
shape = pyhelayers.TTShape([1, 8])
C = encoder.encode_encrypt(shape, M)

print(C)

### Step 2.2. Packing each column of M in a ciphertext
We pack $M$ using tiles (ciphertext) of shape $8 \times 1$.
This effectively places a singe column in every ciphertext.
Since we have 8 slots and each row has only 5 elements we end up with 8-5=3 unused slots in each ciphertext, which we pad with zeros. 
Here, the tile tensor shape is $\left[\frac{5}{8}, 6\right]$ and the external tensor shape is $1 \times 6$.

In [ ]:
shape = pyhelayers.TTShape([8, 1])
cvals = encoder.encode_encrypt(shape, M)

print(cvals)

### Step 2.3. Packing with $2\times 4$ block in each ciphertext
In this example, we pack $M$ using tiles of shape $2 \times 4$.
This effectively divides the matrix to sub-blocks and places each one in a ciphertext.
Since we have 5 rows which is not divisible by 2, the last row of tiles is padded with a row of zeros.
The tile tensor shape is $\left[\frac{5}{2}, \frac{6}{4}\right]$ and the external tensor shape is $3 \times 2$.

In [ ]:
shape = pyhelayers.TTShape([2, 4])
cvals = encoder.encode_encrypt(shape, M)

print(cvals)

### Question 
What shpe yields the minimal number of tiles when the number of slots in a ciphertext is 16?
<details>
    <summary> Answer </summary>

Tile shape 1x16, Number of tiles = 5, Tile tensor shape = [5, 6/16]<br/>
<b>Tile shape 2x8, Number of tiles = 3, Tile tensor shape = [5/2, 6/8]</b><br/>
Tile shape 4x4, Number of tiles = 4, Tile tensor shape = [5/4, 6/4]<br/>
<b>Tile shape 8x2, Number of tiles = 3, Tile tensor shape = [5/8, 6/2]</b><br/>
Tile shape 16x1, Number of tiles = 6, Tile tensor shape = [5/16, 6]<br/><br/>    
</details>

<br/>
What shape yields the minimal number of tiles when the number of slots in a ciphertext is 32? 
<br/>
<details>
    <summary> Answer </summary>
Tile shape 1x32, Number of tiles = 5, Tile tensor shape = [5, 6/32]<br/>
Tile shape 2x16, Number of tiles = 3, Tile tensor shape = [5/2, 6/16]<br/>
<b>Tile shape 4x8, Number of tiles = 2, Tile tensor shape = [5/4, 6/8]</b><br/>
    <b>Tile shape 8x4, Number of tiles = 2, Tile tensor shape = [5/8, 6/4]</b><br/>
Tile shape 16x2, Number of tiles = 3, Tile tensor shape = [5/16, 6/2]<br/>
Tile shape 32x1, Number of tiles = 6, Tile tensor shape = [5/32, 6]    <br/><br/>
</details>

<br/>
Use the code below to test your results.

In [ ]:
# Set s with the required number of slots
s = 16

# Initialize a mockup context with 's' slots. 
requirement2 = pyhelayers.HeConfigRequirement.insecure(
    num_slots = s,                  # Number of slots per ciphertext
    multiplication_depth = 2,       # Allow 2 levels of multiplications
    fractional_part_precision = 40, # Set the precision to 1/2^40.
    integer_part_precision = 20    # Set the largest number to 2^20.
    )

# Use a mockup HE context and initialize it with the requirements object above.
he_context2 = pyhelayers.MockupContext()
he_context2.init(requirement2)

# Create the Encoder using the context.
encoder2 = pyhelayers.TTEncoder(he_context2)

for i in range(int(math.log2(s))+1):
    d1 = 2**i
    d2 = int(s/d1)
    shape2 = pyhelayers.TTShape([d1, d2])
    cvals = encoder2.encode_encrypt(shape2, M)
    print(f'Tile shape {d1}x{d2}, \tNumber of tiles = {cvals.get_num_used_tiles()}, \tTile tensor shape = {cvals.get_shape()}')


## Step 3. Solving a batch of linear regression. 

A linear regression inference operation involves solving the equation $y = x \cdot w$, where $x, w \in \mathbb{R}^d$ and $y \in \mathbb{R}$. When using CKKS, we would like to leverage its single instruction multiple data (SIMD) capabilities. Specifically, we would like to perform a batch of $n$ inference opertions at once.

Let $X$ be the $n\times d$ matrix which is the concatenation of $x_1, \ldots, x_n \in \mathbb{R}^d$ samples. Then,
$(y_1, \ldots, y_n) = X \cdot w \in \mathbb{R}^d$ is the linear regression output.

Let's try a few different packing options. For that, we define the function `linearRegressionBatch` that receives $n$ and $d$ as its input and then perform a linear regression inference operation. The $alpha$ argument determines the log of the first tile size. The plaintext values are dummy values.


In [ ]:
# A packing-oblivious function that get n and d. Then it generates an input matrix X and a weight vector w
# computes y = Xw and returns it
def linearRegressionBatch(n, d, alpha):
    assert isinstance(alpha, int),                  "alpha must be integer"
    assert alpha >= 0,                              "alpha must be non-negative"
    assert pow(2,alpha) <= he_context.slot_count(), "alpha must be smaller than log_2 n"

    he_context.start_operation_count_track()
    
    # Generate the input matrix X and weights vector w (in plaintext)
    X = genDemoMatrix(n,d)
    w = genDemoMatrix(1,d) 

    # Compute and set the tile tensors dimensions
    nDim = pow(2,alpha)
    dDim = he_context.slot_count() / nDim
    shapeX = pyhelayers.TTShape([nDim, dDim])
    shapeW = pyhelayers.TTShape([nDim, dDim]).get_with_duplicated_dim(0)
    
    # Encrypt the input matrix
    cX = encoder.encode_encrypt(shapeX, X)
    cW = encoder.encode_encrypt(shapeW, w)
    
    print("X tensor:\n", cX)
    print("w tensor:\n",cW)

    # Compute and print the inference results
    cY = cX
    cY.multiply_and_sum(cW,1)
    print("y=", cY)
    print(he_context.get_stats_and_clear())
    
    # We return the number of used tiles in Y
    return cY.get_num_used_tiles()

#### Linear regression with alpha=2 (nDim=4)

In [ ]:
num_tiles=linearRegressionBatch(n=6, d=4, alpha=2)
assert num_tiles==2

Notice the second column contains values that seem random. Can you understand what they are? 

<details>
<summary>Hint</summary>
1*1 + 4*0 + 3*3 + 6*2 = 22
</details>

#### Linear regression with alpha=0 (nDim=1)

In [ ]:
num_tiles=linearRegressionBatch(6,4,0)
assert num_tiles==6

#### Linear regression with alpha = 1 (nDim=2)

In [ ]:
num_tiles=linearRegressionBatch(6,4,1)
assert num_tiles==3


## Step 5. Up next

In the above examples, we saw how even for a simple operation like linear regression the different packing have high effect on the number of tiles and thus on the number of operations. 
In the next notebook we are going to see how different packing options affect the performance of our code.